In [1]:
# Bibliotecas
import pandas as pd
import requests
import json


In [2]:
import pandas as pd

# Carregar o CSV
df = pd.read_csv('../dados/AbandonoEscolar_RendaMedia_2013_2023.csv')

# Ver as primeiras linhas
df.head()


,Ano,Unidade_Geografica,Regiao,Localizacao,Dependencia_Administrativa,Grupo_de_Abandono,Taxa_Abandono,Renda_Media
0,2013,Norte,Brasil,Total,Total,Ensino Fundamental,4.0,2252
1,2013,Norte,Brasil,Total,Pública,Ensino Fundamental,4.3,2252
2,2013,Norte,Brasil,Total,Privada,Ensino Fundamental,0.3,2252
3,2013,Norte,Brasil,Total,Municipal,Ensino Fundamental,4.3,2252
4,2013,Norte,Brasil,Total,Federal,Ensino Fundamental,0.0,2252


In [3]:
# Criar variável alvo
df['evasao_alta'] = (df['Taxa_Abandono'] > 20).astype(int)

# Ver distribuição da variável alvo
df['evasao_alta'].value_counts()


evasao_alta
0    89921
1      239
Name: count, dtype: int64

In [4]:
# Selecionar features e target
features = ['Ano', 'Regiao', 'Localizacao', 'Dependencia_Administrativa', 'Renda_Media']
target = 'evasao_alta'

df = df[features + [target]]


In [5]:
df_encoded = pd.get_dummies(df, columns=['Regiao', 'Localizacao', 'Dependencia_Administrativa'], drop_first=True)
df_encoded.head()


,Ano,Renda_Media,evasao_alta,Regiao_Centro-Oeste,Regiao_Nordeste,Regiao_Norte,Regiao_Sudeste,Regiao_Sul,Localizacao_Total,Localizacao_Urbana,Dependencia_Administrativa_Federal,Dependencia_Administrativa_Municipal,Dependencia_Administrativa_Privada,Dependencia_Administrativa_Pública,Dependencia_Administrativa_Total
0,2013,2252,0,False,False,False,False,False,True,False,False,False,False,False,True
1,2013,2252,0,False,False,False,False,False,True,False,False,False,False,True,False
2,2013,2252,0,False,False,False,False,False,True,False,False,False,True,False,False
3,2013,2252,0,False,False,False,False,False,True,False,False,True,False,False,False
4,2013,2252,0,False,False,False,False,False,True,False,True,False,False,False,False


In [6]:
from sklearn.model_selection import train_test_split

X = df_encoded.drop('evasao_alta', axis=1)
y = df_encoded['evasao_alta']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [7]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report

# Treinamento
model = GradientBoostingClassifier()
model.fit(X_train, y_train)

# Predição e avaliação
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00     17988
           1       0.00      0.00      0.00        44

    accuracy                           1.00     18032
   macro avg       0.50      0.50      0.50     18032
weighted avg       1.00      1.00      1.00     18032



C:\Users\amand\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\amand\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\amand\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\sklearn\metrics\_classificati

In [8]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

print("Antes:", y_train.value_counts().to_dict())
print("Depois:", pd.Series(y_resampled).value_counts().to_dict())


Antes: {0: 71933, 1: 195}
Depois: {0: 71933, 1: 71933}


In [9]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix

# Treinar com os dados balanceados
model = GradientBoostingClassifier(random_state=42)
model.fit(X_resampled, y_resampled)

# Fazer predições no conjunto de teste
y_pred = model.predict(X_test)

# Avaliação
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))


              precision    recall  f1-score   support

           0       1.00      0.91      0.95     17988
           1       0.02      0.59      0.03        44

    accuracy                           0.91     18032
   macro avg       0.51      0.75      0.49     18032
weighted avg       1.00      0.91      0.95     18032

[[16298  1690]
 [   18    26]]
